# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [3]:
# Your code here; create the connection
conn = sqlite3.Connection("data/data.sqlite")


## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [9]:
# Your code here
q0 = """
SELECT
    c.customerNumber,
    c.contactLastName,
    c.contactFirstName
FROM customers c, orders o
    WHERE o.customerNumber = c.customerNumber
AND orderDate = '2003-01-31'
;
"""
pd.read_sql(q0, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [17]:
# Your code here
q0 = """
SELECT
p.productCode, p.productName,
sum(od.quantityOrdered) as number_sold,
count(orderNumber) as number_orders

from products p, orderDetails od
where p.productCode = od.productCode
group by od.productCode
order by number_sold desc
;
"""
pd.read_sql(q0, conn)


,productCode,productName,number_sold,number_orders
0,S18_3232,1992 Ferrari 360 Spider red,1808,53
1,S18_1342,1937 Lincoln Berline,1111,28
2,S700_4002,American Airlines: MD-11S,1085,28
3,S18_3856,1941 Chevrolet Special Deluxe Cabriolet,1076,28
4,S50_1341,1930 Buick Marquette Phaeton,1074,28
...,...,...,...,...
104,S18_2870,1999 Indy 500 Monte Carlo SS,855,25
105,S18_2248,1911 Ford Town Car,832,25
106,S24_3969,1936 Mercedes Benz 500k Roadster,824,25
107,S24_1046,1970 Chevy Chevelle SS 454,803,25


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [36]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q0 = """
SELECT
count(distinct o.customerNumber) as "How many people ordered this", p.productName, p.productCode
from products p, orderDetails od, orders o, customers c
where p.productCode = od.productCode and
od.orderNumber = o.orderNumber and
c.customerNumber = o.customerNumber
group by p.productCode
;
"""
pd.read_sql(q0, conn)

,How many people ordered this,productName,productCode
0,26,1969 Harley Davidson Ultimate Chopper,S10_1678
1,27,1952 Alpine Renault 1300,S10_1949
2,26,1996 Moto Guzzi 1100i,S10_2016
3,25,2003 Harley-Davidson Eagle Drag Bike,S10_4698
4,27,1972 Alfa Romeo GTA,S10_4757
...,...,...,...
104,22,The Titanic,S700_3505
105,24,The Queen Mary,S700_3962
106,26,American Airlines: MD-11S,S700_4002
107,27,Boeing X-32A JSF,S72_1253


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [53]:
# Your code here
q0 = """
select How_many_people_ordered_this, employeeNumber, firstName, lastName, city, officeCode from 

(SELECT
count(distinct o.customerNumber) as How_many_people_ordered_this, p.productName, p.productCode, c.salesRepEmployeeNumber,
e.employeeNumber, e.firstName, e.lastName, o.city, e.officeCode
from employees e, offices o, 
products p, orderDetails od, orders o, customers c
where p.productCode = od.productCode and
c.salesRepEmployeeNumber = e.employeeNumber and
e.officeCode = o.officeCode and
od.orderNumber = o.orderNumber and
c.customerNumber = o.customerNumber
group by p.productCode)

where How_many_people_ordered_this < 20

;
"""
pd.read_sql(q0, conn)

,How_many_people_ordered_this,employeeNumber,firstName,lastName,city,officeCode
0,18,1501,Larry,Bott,London,7
1,19,1370,Gerard,Hernandez,Paris,4
2,18,1501,Larry,Bott,London,7
3,18,1501,Larry,Bott,London,7
4,18,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [61]:
# Your code here
q0 = """
SELECT
e.employeeNumber, e.firstName, e.lastName, count(c.customerNumber), avg(c.creditLimit)
from employees e, customers c
where 
c.salesRepEmployeeNumber = e.employeeNumber 
group by e.employeeNumber
;
"""
pd.read_sql(q0, conn)


,employeeNumber,firstName,lastName,count(c.customerNumber),avg(c.creditLimit)
0,1165,Leslie,Jennings,6,100433.333333
1,1166,Leslie,Thompson,6,65266.666667
2,1188,Julie,Firrelli,6,73916.666667
3,1216,Steve,Patterson,6,81533.333333
4,1286,Foon Yue,Tseng,7,66614.285714
5,1323,George,Vanauf,8,80887.500000
6,1337,Loui,Bondur,6,86233.333333
7,1370,Gerard,Hernandez,7,91785.714286
8,1401,Pamela,Castillo,10,81340.000000
9,1501,Larry,Bott,8,91187.500000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!